In [1]:
import json
import socket
import time 

from dotenv import dotenv_values
import requests as  rq

from confluent_kafka import Producer

In [2]:
key = dotenv_values("./.env")

In [3]:
url_id_excange = "https://api.coingecko.com/api/v3/exchanges"

headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": key["API_KEY"]
}

response_id = rq.get(url_id_excange, headers=headers)

if response_id.status_code == 200:

    response_id = response_id.json()

    id_exchange: dict = {}
    for json_ in response_id:
        id_exchange[json_.get("id")] = json_.get("name")
else:
    print(f"Status code: {response_id.status_code}")

In [4]:
def get_volume_day(excange_id: str) -> dict | None:
    url = f"https://api.coingecko.com/api/v3/exchanges/{excange_id}/volume_chart?days=1"
    
    response = rq.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"excange_id ({excange_id}) - Status code: {response.status_code}")
        return

def generator_exchange_volume():
    for exchange in id_exchange:
        volume_day = get_volume_day(exchange)
        
        if volume_day is None:
            volume_day = []
            
        data = {"exchange": id_exchange.get(exchange), "volume_day": volume_day}
        yield str.encode(json.dumps(data))

In [5]:
topic = "module3"

config = {
    "bootstrap.servers": "localhost:9092",
    "client.id": socket.gethostname()
}

producer = Producer(config)

In [6]:
for data in generator_exchange_volume():
    producer.produce(topic, value=data)
    time.sleep(2)
    producer.flush()

exit_message = str.encode(json.dumps({"exit": True}))
producer.produce(topic, value=exit_message)
producer.flush()

0